# Lab 5

## RNA Folding Simulation

### This lab focuses on the creation of a 3D RNA structure from an input of a nucleotide sequence using a predictive algorithm from the RNAfold library. 


Sequences pulled from:
https://www.nature.com/articles/nbt1155

## Part 1
Getting sequence data from figures YES-1 in 2A, NOT-1 in 4A, AND-1 in 5A, and OR-1 in 6A

In [1]:
yes='GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC'
no='GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGACGAAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC'
an='GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACAUGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU'
or1='GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC'
total=[yes,no,an,or1]

Make a table with name, start/end of OBS-1, start/end of OBS-2 (NA if none), and start/end of red regions

In [2]:
import pandas as pd
import subprocess

In [3]:
d={'Name':['YES-1','NOT-1','AND-1','OR-1'],
  'OBS-1 Start-End':['26-48','44-66','30-45','27-46'],
  'OBS-2 Start-End':[None,None,'49-64','47-66'],
  'Red Start-End':['16-21;49-54','40-43;74-77','16-23;70-77','16-26;67-77']}

In [4]:
riboswitch=pd.DataFrame(data=d)

In [5]:
riboswitch

,Name,OBS-1 Start-End,OBS-2 Start-End,Red Start-End
0,YES-1,26-48,None,16-21;49-54
1,NOT-1,44-66,None,40-43;74-77
2,AND-1,30-45,49-64,16-23;70-77
3,OR-1,27-46,47-66,16-26;67-77


# Part 2
Testing RNAfold and RNAplot in terminal to see if output works...
![my_image](images/scr.png)

after confirming that it works in terminal, will create a python function to automate the process using subprocess

In [8]:
def rna(sequence,c=0):
    #take in a rna sequence, and create a rna.ps file in the working directory
    
    if c != 0:
        command=['RNAfold', '--constraint']
        entire = sequence + '\n' + c
    else:
        command=['RNAfold']
        entire = sequence
    p=subprocess.run(command,
                    input=bytes(entire,'ascii'),
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)
    print(p.stderr.decode())
    subprocess.run(['RNAplot','-o','svg'],
                  input=bytes(p.stdout.decode(),'ascii'),
                  stderr=subprocess.PIPE)
    return 

In [9]:
rna(yes)

### YES1
![yes1](images/yes1.png)

This does not look exactly like what was obtained in the paper, as the paper had three main stems rather than the four in this model.Stem 1 is the same, Stem III is the same, but Stem IV in the paper has been turned into two stems in this figure. The temperature used was 37&deg;C, with a free energy value based on the partition function at -35.6 $kcal$ $*$ $mol^{-1}$.

The Red regions are also not overlapping in this structure, just as they aren't in the paper.

One possible reason for this is that the inactive structures were computed to be less stable than their active structues, while this algorithm is simply finding the most stable conformation from the sequence given. 

In [10]:
rna(no)

### Not1
![not1](images/not1.svg)

This structure looks very similar to that obtained in the paper. This structure has the correct number of stems and the stems have the same base pairings, as far as I can tell. The only thing that looks different is the core, but they are actually the same, and that this rendering is just different in appearance than the one in the paper. The RED regions of the RNA are also correctly base-paired, as they should be.

This makes sense because we would expect that this riboswitch should be more stable in the active, unbound state than it is in the inactive, bound state. And since this algorithm calculates the most stable state, it would correctly calculate this one. This is the opposite of the YES-1, where it was less stable in its unbound, inactive state. As far as I can tell, the temp used is 37&deg;C also, but the free energy value is now -30.55 $kcal$ $*$ $mol^{-1}$.

In [11]:
rna(an)

### AND-1
![my_image](images/and1.svg)

This structure looks very similar to that obtained in the paper. This structure has the correct number of stems and the stems have the same base pairings. Because this should be inactive, the RED regions should not base pair with each other, which they don't.

The paper says that this inactive state is energetically more favorable than the on state or the transition states at 37&deg;C, so we should expect this algorithm to have correctly found the structure by computing the most stable structure from the sequence.

In [12]:
rna(or1)

### OR-1
![my_image](images/or1.svg)

This structure is also the same as the structure predicted in the paper for OR-1. They both have the same stems and same base pairs bonded with each other. One of the base pairs from the RED regions are bonded to each other, just as described in the paper.
    
This also makes sense because this is the most stable state of the protein, being more stable than the on state at 37&deg;C

## Part 3
Look at structure of sequences when OBS is bound

In [13]:
cons = []
for i in [0,1]:
    lower = int(riboswitch['OBS-1 Start-End'][i].split('-')[0])-1
    upper = int(riboswitch['OBS-1 Start-End'][i].split('-')[1])-1
    cseq = ''
    print(lower)
    for s in range(len(total[i])):
        #print(s)
        if s >= lower and s <= upper:
            cseq+='x'
        else:
            cseq+='.'
    print(cseq)
    cons.append(cseq)

25
.........................xxxxxxxxxxxxxxxxxxxxxxx................................
43
...........................................xxxxxxxxxxxxxxxxxxxxxxx..........................


In [14]:
alland=[]
lower = int(riboswitch['OBS-1 Start-End'][2].split('-')[0])-1
upper = int(riboswitch['OBS-1 Start-End'][2].split('-')[1])-1
lower2 = int(riboswitch['OBS-2 Start-End'][2].split('-')[0])-1
upper2 = int(riboswitch['OBS-2 Start-End'][2].split('-')[1])-1
for i in range(3):
    cseq = ''
    for s in range(len(an)):
        if i==0:
            cond = s >= lower and s <= upper
        elif i==1:
            cond = s >= lower2 and s <= upper2
        elif i==2:
            cond = (s >= lower and s <= upper) or (s >= lower2 and s <= upper2)
        else:
            break
            
        if cond:
            cseq+='x'
        else:
            cseq+='.'
    alland.append(cseq)
    print(cseq)
    
allor=[]
lower = int(riboswitch['OBS-1 Start-End'][3].split('-')[0])-1
upper= int(riboswitch['OBS-1 Start-End'][3].split('-')[1])-1
lower2 = int(riboswitch['OBS-2 Start-End'][3].split('-')[0])-1
upper2 = int(riboswitch['OBS-2 Start-End'][3].split('-')[1])-1
for i in range(3):
    cseq = ''
    for s in range(len(or1)):
        if i==0:
            cond = s >= lower and s <= upper
        elif i==1:
            cond = s >= lower2 and s <= upper2
        elif i==2:
            cond = (s >= lower and s <= upper) or (s >= lower2 and s <= upper2)
        else:
            break
            
        if cond:
            cseq+='x'
        else:
            cseq+='.'
    allor.append(cseq)
    print(cseq)

.............................xxxxxxxxxxxxxxxx...................................................................
................................................xxxxxxxxxxxxxxxx................................................
.............................xxxxxxxxxxxxxxxx...xxxxxxxxxxxxxxxx................................................
..........................xxxxxxxxxxxxxxxxxxxx.........................................................
..............................................xxxxxxxxxxxxxxxxxxxx.....................................
..........................xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.....................................


In [17]:
rna(yes,cons[0])

### YES-1 OBS
![my_image](images/yes1-1.svg)

This sequence is the same as the one described in the paper, and now this folding structure is correct because the paper describes that this structure is more stable than the inactive state, so the algorithm would automatically generate this as the best fit for the RNA. The RED regions of the RNA are also now base-paired, as they should be

In [18]:
rna(no,cons[1])

### NOT-1 OBS
![my_image](images/not1-1.svg)

This structure is also the same as the OBS bound structure described in the paper for NOT-1. The RED regions are not lined up, and this makes sense because this structure should be inactive.

### AND-1 OBS (three possibilities)

In [19]:
rna(an,alland[0])

#### OBS 1

![my_image](images/and1-1.svg)

OBS 1 is not the exact same as the structure described in the paper as it is missing some base pairing that the structure paper describes, but the RED regions of the bases are not overlapping in this structure. There is also an extra stem near the bottom. In this state, it should be inactive.

In [20]:
rna(an,alland[1])

#### OBS 2

![my_image](images/and1-2.svg)

This structure is also not the exact same as the structure described in the paper for binding at OBS-2, evident through the appearance of the stem closest to the unbound OBS-2 region. However, the RED regions are still not overlapping here, so there should be no activity also.

In [21]:
rna(an,alland[2])

### OBS 1 and 2
![my_image](images/and1-12.svg)

This structure is also not the same as the structure depicted in the paper. For one, the RED regions do not overlap in this structure. This means that based on the algorithm that we used to calculate structure, this riboswitch should not work as an AND gate because of the lack of overlap of RED regions. We might have obtained different results because this structure is less stable than the structure with unbound OBS sites, or because of other paramaters they used to create the RNA structure. 

### OR-1 (Three Possibilities)

In [22]:
rna(or1,allor[0])

#### OBS 1
![my_image](images/or1-1.svg)

This structure is different from the structure described in the paper. The RED groups do not align with each other, which means that it would be inactive when it should be active

In [23]:
rna(or1,allor[1])

#### OBS 2
![my_image](images/or1-2.svg)

This structure is the same as the structure described in the paper. As such, the RED regions are overlapping. This implies that this structure should be active, as it should be if this is an OR gate.

In [24]:
rna(or1,allor[2])

#### OBS 1 and OBS 2
![my_image](images/or1-12.svg)

This structure is the same as the OBS-2 structure, and because of this it is also the same as the paper, since all of the OR gates with a OBS bound should look the same. Because of this, the RED regions are bonded together, making this an active ribozyme that can cleave. This should be true if this is an OR gate

## Part 4

From the data gathered so far, we can conclude that for the AND-1 gate:

F/F = F, T/F = F, F/T = F, and T/T = F

and for the OR-1 gate:

F/F = F, T/F = F, F/T = T, and T/T = T

where the first entry is OBS1 and the second is OBS2

We can display this in a logic table as such...

In [131]:
l = {'OBS1 bound':['F','T','F','T'],
    'OBS2 bound':['F','F','T','T'],
    'AND-1 Active':['F','F','F','F'],
    'OR-1 Active':['F','F','T','T']}
df = pd.DataFrame(data=l)
cols = df.columns.tolist()
cols = cols[1:] + [(cols[0])]
logic = df[cols]
logic

,OBS1 bound,OBS2 bound,OR-1 Active,AND-1 Active
0,F,F,F,F
1,T,F,F,F
2,F,T,T,F
3,T,T,T,F


Based on my results, this suggests that the AND and OR gates do not function as they are described in the paper. This is because the AND gate is not active (the RED regions do not bond) when both OBS sites are bound, and because the OR gate is not active (the RED regions do not bond) when the OBS1 site is bound but the OBS2 site isn't. Again, this could be because of different parameters that were used in the paper to calculate these structures. 